In [ ]:
from ipyprogressivis.widgets import ControlPanel, ModuleGraph
from ipyprogressivis.widgets.scatterplot import Scatterplot
sc = Scatterplot()
gr = ModuleGraph()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import time
import pandas as pd
import copy
from progressivis.core.api import Scheduler, Sink, JSONEncoderNp
from progressivis.table.api import PTable
from progressivis.vis import MCScatterPlot
from progressivis.io.api import CSVLoader
from progressivis.table.constant import Constant

def _quiet(x): pass


def _filter(df):
    pklon = df['pickup_longitude']
    pklat = df['pickup_latitude']
    dolon = df['dropoff_longitude']
    dolat = df['dropoff_latitude']

    return df[(pklon > -74.08) & (pklon < -73.5) & (pklat > 40.55) & (pklat < 41.00) &
                  (dolon > -74.08) & (dolon < -73.5) & (dolat > 40.55) & (dolat < 41.00)]
    


PREFIX = "https://www.aviz.fr/nyc-taxi/"
SUFFIX = '.bz2'

URLS = [
    PREFIX+'yellow_tripdata_2015-01.csv'+SUFFIX,
    PREFIX+'yellow_tripdata_2015-02.csv'+SUFFIX,
    PREFIX+'yellow_tripdata_2015-03.csv'+SUFFIX,
    PREFIX+'yellow_tripdata_2015-04.csv'+SUFFIX,
    PREFIX+'yellow_tripdata_2015-05.csv'+SUFFIX,
    PREFIX+'yellow_tripdata_2015-06.csv'+SUFFIX,
]

FILENAMES = pd.DataFrame({'filename': URLS})
CST = Constant(PTable('filenames', data=FILENAMES))
CSV = CSVLoader(skipinitialspace=True,
                usecols=['pickup_longitude', 'pickup_latitude',
                             'dropoff_longitude', 'dropoff_latitude'],
                filter_=_filter) 
CSV.input.filenames = CST.output[0]
sink = Sink()
sink.input.inp = CSV.output[0]

MULTICLASS = MCScatterPlot(classes=[
    ('pickup', 'pickup_longitude', 'pickup_latitude'),
    ('dropoff', 'dropoff_longitude', 'dropoff_latitude')], approximate=True)
MULTICLASS.create_dependent_modules(CSV, 'result')
sc.link_module(MULTICLASS)
cpanel = ControlPanel(CSV.scheduler)


In [ ]:
import ipywidgets as ipw
import asyncio as aio
CSV.scheduler.task_start()
await aio.sleep(1)
json_ = CSV.scheduler.to_json(short=False)
json_str = JSONEncoderNp.dumps(json_, skipkeys=True)
gr.data = json_str
tab = ipw.Tab()
tab.children = [sc, gr]
tab.set_title(0, 'Scatterplot')
tab.set_title(1, 'Module graph')
ipw.VBox([tab, cpanel])